In [1]:
    !pwd
%matplotlib inline

/home/jamnic/programming/handson-ml3/test_notebooks


In [2]:
import os
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

import matplotlib.pyplot as plt

In [3]:
data_dir = os.path.join(os.getcwd(), '..', 'datasets', 'titanic')
train_data = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test_data = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Embarked', 'Cabin'], axis='columns', inplace=True)
train_data['Sex'].replace({'male': 1, 'female': 0}, inplace=True)
train_data['Age'].fillna(train_data['Age'].dropna().mean(), inplace=True)

In [6]:
train_set, val_set = train_test_split(train_data, train_size=0.8)

X_train = train_set.drop(['Survived'], axis=1)
X_val = val_set.drop(['Survived'], axis=1)

y_train = train_set['Survived']
y_val = val_set['Survived']

In [7]:
survival_percentage = ((y_train == 1).sum() / len(y_train)) * 100
print(f'Survival Percentage: {round(survival_percentage, 1)}%')

Survival Percentage: 40.0%


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC

In [9]:
lin_reg = LinearRegression()
lin_reg_model = lin_reg.fit(X_train, y_train)

In [10]:
test_data.drop(['PassengerId', 'Name', 'Ticket', 'Embarked', 'Cabin'], axis='columns', inplace=True)
test_data['Sex'].replace({'male': 1, 'female': 0}, inplace=True)
test_data['Age'].fillna(X_train['Age'].dropna().mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].dropna().mean(), inplace=True)

In [11]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('fit', SVC())
])

model = pipeline.fit(X_train, y_train)

In [12]:
predictions = model.predict(X_val)

In [13]:
accuracy_score(y_val, predictions)

0.8324022346368715

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100], 
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

grid_search = GridSearchCV(SVC(), param_grid, cv=5, verbose=2)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.2s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=   0.1s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END ....................C=0.1, gamma=scale, kernel=poly; total time=   0.0s
[CV] END ....................C=0.1, gamma=scale